Regression model

In [2]:
import sklearn
print(sklearn.__version__)
from sklearn.tree import DecisionTreeRegressor
import pandas as pd

0.24.2


In [3]:
df = pd.read_csv('mv_port_processed2.csv')

# typical cleaning
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Project Number'] = df['Project Number'].astype('str') 
df = df.loc[:, ~df.columns.str.contains('^index')]

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (17,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df['Project Status'].unique()

array(['Open/Active', 'On Hold - Inactive', 'Closed',
       'Closed-Open to Collect', 'Final Closure Requested', 'Unspecified',
       'New - Pending Approval', 'Rejected', 'New', 'Manual_Adj'],
      dtype=object)

In [5]:
df_sub = df[['Actual Net Revenue - MTD', 'Date', 'Project Number', 'Project Status', 'Contract Type Name', 'Ultimate Customer AUS Market Sector', 'Division Short Name', 'Proj_CVP', 'Proj_Serv_Offering',
             'primary_client_industry', 'Funding Amount - ITC (Snapshot)', 'Backlog NR',  'Actual Direct Labor Cost - MTD', 'Actual Non-Labor Cost - MTD',
       'Actual Gross Revenue - MTD', 'EBITA - MTD',
       'Forecast Direct Labor Cost - ITC', 'Forecast Net Revenue - ITC',
       'Order Intake (NR) - MTD', 'AUS Mkt Sector Aggregate', 'Baseline NR', 'Organization Size',
       'Risk Profile', 'riskrating_project', 'ActualNR_sum', 'Projectduration', 'Completion', 'scheduled_duration', 'Scheduled_projectduration', 'intime', 'seasonality_month']]

In [6]:
Project_Status = pd.get_dummies(df_sub['Project Status'])
Contract_Name = pd.get_dummies(df_sub['Contract Type Name'])
Ultimate_AUS = pd.get_dummies(df_sub['Ultimate Customer AUS Market Sector'])
Divison = pd.get_dummies(df_sub['Division Short Name'])
#Region = pd.get_dummies(df_sub['Region'], drop_first=True)
Proj_CVP = pd.get_dummies(df_sub['Proj_CVP'])
Proj_Serv_Offering = pd.get_dummies(df_sub['Proj_Serv_Offering'])
Client = pd.get_dummies(df_sub['primary_client_industry'])
Aggregate_AUS = pd.get_dummies(df_sub['AUS Mkt Sector Aggregate'])
Orgsize = pd.get_dummies(df_sub['Organization Size'])
Profile = pd.get_dummies(df_sub['Risk Profile'])



df_sub = pd.concat([df_sub, Project_Status], axis=1)
df_sub = pd.concat([df_sub, Contract_Name], axis=1)
df_sub = pd.concat([df_sub, Ultimate_AUS], axis=1)
df_sub = pd.concat([df_sub, Divison], axis=1)
#df_sub = pd.concat([df_sub, Region], axis=1)
df_sub = pd.concat([df_sub, Proj_CVP], axis=1)
df_sub = pd.concat([df_sub, Proj_Serv_Offering], axis=1)
df_sub = pd.concat([df_sub, Client], axis=1)
df_sub = pd.concat([df_sub, Aggregate_AUS], axis=1)
df_sub = pd.concat([df_sub, Orgsize], axis=1)
df_sub = pd.concat([df_sub, Profile], axis=1)

df_sub = df_sub.drop(columns=['Project Status', 'Contract Type Name', 'Ultimate Customer AUS Market Sector', 'Division Short Name',
                              'Proj_CVP', 'Proj_Serv_Offering', 'primary_client_industry', 'AUS Mkt Sector Aggregate', 'Organization Size', 'Risk Profile'])
df_sub

,Actual Net Revenue - MTD,Date,Project Number,Funding Amount - ITC (Snapshot),Backlog NR,Actual Direct Labor Cost - MTD,Actual Non-Labor Cost - MTD,Actual Gross Revenue - MTD,EBITA - MTD,Forecast Direct Labor Cost - ITC,...,Public,Transp/Log,Key Global,Key Regional,Local,Unspecified,High,Low,Medium,Unspecified
0,111.72,201711,000221760018,68313,0.00,46.65,0.0,111.72,-5.17,0.00,...,0,0,0,1,0,0,0,0,0,1
1,10406.45,201712,000221760018,68313,0.00,1353.76,0.0,52149.10,7014.47,0.00,...,0,0,0,1,0,0,0,0,0,1
2,-2048.91,201801,000221760018,68313,0.00,17.55,0.0,-2048.91,-2092.03,0.00,...,0,0,0,1,0,0,0,0,0,1
3,87.02,201802,000221760018,68313,3207.95,36.55,0.0,14893.44,-2.78,2064.61,...,0,0,0,1,0,0,0,0,0,1
4,185.22,201803,000221760018,68313,3022.73,85.81,0.0,185.22,-25.60,2064.61,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266972,0.00,201902,zz000zzznt20,0,0.00,0.00,0.0,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,1
1266973,0.00,201903,zz000zzznt20,0,0.00,0.00,0.0,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,1
1266974,0.00,201904,zz000zzznt20,0,0.00,0.00,0.0,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,1
1266975,0.00,201905,zz000zzznt20,0,0.00,0.00,0.0,0.00,0.00,0.00,...,0,0,0,0,1,0,0,0,0,1


In [7]:
''' Now we cant forecast for multiple projects a the same time, but there are different options to input the data'''


''' we can either use the previous time periods of one project'''


def random_project(df):
    df = df.sample(n=1)
    project_number = df['Project Number'].item()
    return project_number

#larger dataframe including shifted backlog
def obtain_df(project, df):
    df_project = df[df['Project Number'] == project]
    #df_project = df_project[['Fiscal Period Code', 'Funding Amount - ITC (Snapshot)', 'Backlog NR', 'Actual Gross Revenue - MTD']].copy()
    df_project.pop('Project Number')
    df_project.reset_index(inplace=True)
    df_project = df_project.loc[:, ~df_project.columns.str.contains('^index')]
    return df_project

''' we could use several projects without any order, or can we introduce order'''


' we could use several projects without any order, or can we introduce order'

In [8]:
import numpy as np
from numpy import inf

In [9]:
''' Now we need to create the zero padding'''
### The idea is that we outer merge the dataframe with the whole time series. The gaps will then be filled filled

def padding(maindf, tragetdf):
    dates = sorted(maindf['Date'].unique())
    dates = pd.DataFrame(dates, columns=['Date'])
    #now we merge and pad the resulting dataframe
    tragetdf=pd.merge(dates, tragetdf, how='outer', on='Date')
    tragetdf['Timeperiod'] = tragetdf.pop('Date')
    tragetdf[['Closed']] = tragetdf[['Closed']].fillna(value=1)
    tragetdf.fillna(0, inplace=True)
    tragetdf=tragetdf[~tragetdf.isin([np.nan, np.inf, -np.inf]).any(1)]
    return tragetdf

In [10]:
x=random_project(df_sub)

df_sample=obtain_df(x, df_sub)
df_sample=padding(df, df_sample)
print(x)

30001817


In [11]:
'''
from sklearn.preprocessing import StandardScaler  


scaler = StandardScaler()
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

'''

'\nfrom sklearn.preprocessing import StandardScaler  \n\n\nscaler = StandardScaler()\nscaler.fit(X_train)  \nX_train = scaler.transform(X_train)  \nX_test = scaler.transform(X_test)  \n\n'

In [12]:
from sklearn.model_selection import train_test_split

X = df_sample.iloc[:,1:]
y = df_sample.iloc[:,:1]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False) #we do a 20% test split, which is equal to 12 observations
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(50, 245) (13, 245) (50, 1) (13, 1)


In [14]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train.values.ravel())
print(model.score(X_test, y_test))

-0.08333333333333326


In [16]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


yhat=model.predict(X_test)
r2_score(yhat, y_test)


0.0

In [18]:
mse=mean_squared_error(yhat, y_test)

In [19]:
import numpy as np
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))
print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse), 2)))
print('The normalized Root Mean Squared Error of our forecasts is {}'.format( ((round(np.sqrt(mse), 2))/ (yhat.max() - yhat.min())) * 100))


The Mean Squared Error of our forecasts is 0.0
The Root Mean Squared Error of our forecasts is 0.01
The normalized Root Mean Squared Error of our forecasts is inf


/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.


In [20]:
df_sample

,Actual Net Revenue - MTD,Funding Amount - ITC (Snapshot),Backlog NR,Actual Direct Labor Cost - MTD,Actual Non-Labor Cost - MTD,Actual Gross Revenue - MTD,EBITA - MTD,Forecast Direct Labor Cost - ITC,Forecast Net Revenue - ITC,Order Intake (NR) - MTD,...,Transp/Log,Key Global,Key Regional,Local,Unspecified,High,Low,Medium,Unspecified,Timeperiod
0,14777.06,850745,86663.52,4451.87,0.0,15539.36,2603.42,169080.92,548149.39,0.00,...,0,0,0,1,0,0,1,0,0,201601
1,22621.81,850745,92028.10,6812.07,0.0,34609.33,3994.20,177895.06,576135.78,27986.39,...,0,0,0,1,0,0,1,0,0,201602
2,33398.32,850745,83949.35,10838.04,0.0,37093.16,3761.70,185295.33,601455.35,25319.57,...,0,0,0,1,0,0,1,0,0,201603
3,26964.36,850745,59838.99,8701.38,0.0,28887.45,3170.44,186389.43,604309.35,2854.00,...,0,0,0,1,0,0,1,0,0,201604
4,10875.28,850745,48963.71,3738.49,0.0,10983.28,652.38,186389.43,604309.35,0.00,...,0,0,0,1,0,0,1,0,0,201605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.00,850746,0.00,0.00,0.0,0.00,0.00,212284.86,685155.58,0.00,...,0,0,0,1,0,0,1,0,0,202011
59,0.00,850746,0.00,0.00,0.0,0.00,0.00,212284.86,685155.57,0.00,...,0,0,0,1,0,0,1,0,0,202012
60,-0.02,850746,0.00,0.00,0.0,-0.02,-0.02,212284.86,685155.57,-0.02,...,0,0,0,1,0,0,1,0,0,202101
61,0.00,850746,0.00,0.00,0.0,0.00,0.00,212284.86,685155.57,0.00,...,0,0,0,1,0,0,1,0,0,202102


In [ ]:
''' Now lets create a pipeline where we predict all models'''

In [ ]:
#Random Forest regressor
# try log transformation


In [29]:
MSE = []
RMSE = []
nRMSE = []
rsquare = [] 


for i in range(500):
    try:
        x=random_project(df_sub)
        df_sample=obtain_df(x, df_sub)
        df_sample=padding(df, df_sample)

        X = df_sample.iloc[:,1:]
        y = df_sample.iloc[:,:1]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
        model = DecisionTreeRegressor()
        model.fit(X_train, y_train.values.ravel())
        #print(model.score(X_test, y_test))

        yhat=model.predict(X_test)
        r2_score(yhat, y_test)
        mse=mean_squared_error(yhat, y_test)

        if yhat.max() == 0:
            yhatmax = 1

        else:
            yhatmax = yhat.max()

        MSE.append(round(mse, 2))
        RMSE.append(round(np.sqrt(mse), 2))
        norm = ((round(np.sqrt(mse), 2))/ (yhatmax - yhat.min()) * 100)
        if norm != inf:
            nRMSE.append(norm)
        
        rsq=r2_score(yhat, y_test)
        rsquare.append(rsq)
        
    except:
        print('fail')

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars
/Users

In [30]:
from statistics import mean
mean(rsquare)

-6.959841820254282e+29

In [31]:
mean(nRMSE)

53798.93534527027

In [32]:
len(df)

1266977